# Football World Cup 2022

In [95]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [96]:
dict_table = pd.read_html("https://www.wallchart.io/football/world-cup-2022.html")
df_matches = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_matches.csv")
df_fixture = pd.read_csv("/Users/hilmanyusoh/Documents/PWFWCup 2022/clean_fifa_worldcup_fixture.csv")

In [110]:
dict_table[2]

,Team,W,D,L,GF - GA,Pts
0,Senegal,0,0,0,0 - 0,0
1,Qatar,0,0,0,0 - 0,0
2,Ecuador,0,0,0,0 - 0,0
3,Netherlands,0,0,0,0 - 0,0


# 1.Calculate team strenght

In [55]:
# split dataframe into datahome and dataway
data_home = datamatches[["HomeTeam","HomeGoals","AwayGoals"]]
data_away = datamatches[["AwayTeam","HomeGoals","AwayGoals"]]

In [56]:
# rename columns
data_home = data_home.rename(columns={"HomeTeam": "Team", "HomeGoals": "GoalsScored", "AwayGoals": "GoalsConceded"})
data_away = data_away.rename(columns={"AwayTeam": "Team", "HomeGoals": "GoalsConceded", "AwayGoals": "GaolsScored"})
                         

In [57]:
# concat data_home and data_away, group by team and calculate the mean
df_team_strenght = pd.concat([data_home, data_away], ignore_index=True).groupby('Team').mean()
df_team_strenght

,GoalsScored,GoalsConceded,GaolsScored
Team,,,
Algeria,0.833333,1.461538,1.142857
Angola,0.000000,0.666667,0.500000
Argentina,1.913793,1.148148,1.130435
Australia,1.000000,1.937500,0.666667
Austria,2.285714,1.620690,0.733333
...,...,...,...
Uruguay,2.060606,1.321429,0.826087
Wales,2.000000,0.800000,0.500000
West Germany,2.500000,1.241935,1.500000


## 2 Function predict_point

In [58]:
def predict_points(home, away):
    if home in df_team_strenght.index and away in df_team_strenght.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strenght.at[home,'GoalsScored'] * df_team_strenght.at[away,'GoalsConceded']
        lamb_away = df_team_strenght.at[away,'GoalsScored'] * df_team_strenght.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goal home team
            for y in range(0,11): #number of goal away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(x, lamb_away)
                if x==y:
                    prob_draw +=p
                elif x > y:
                    prob_home +=p
                else:
                    prob_away +=p
                    
        point_home = 3 * prob_home + prob_draw
        point_away = 3 * prob_away + prob_draw
        return (point_home, point_away)
    else:
        return (0, 0)
    
                

## 2.1 Testing function

In [59]:
# Test with matches: Argentina - Mexico, England - United States, Qatar (H) - Equador
predict_points('Argentina', 'Mexico')
predict_points('England', 'United States')
predict_points('Qatar (H)', 'Equador')

(0, 0)

# 3 Predicting World Cup

## 3.1 Group stage

In [60]:
# splitting fixture into group, knockout, quarter, ...
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [61]:
df_fixture_group_48

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [111]:
# run all the matches in the group stage and update group tables
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].Values
    df_fixture_group_6 = df_fixture_48[df_fixture_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
dict_table[group] = dict_table[group][['Team', 'Pts']]
dict_table[group] = dict_table[group].round(0)

TypeError: list indices must be integers or slices, not DataFrame